In [1]:
import numpy as np
import pandas as pd
import re

### Enrichment ideas: import other categories:

In [113]:
def matched_string(pattern,string):
    result = re.match(pattern,string)
    if result: return(string)
    else: return("")

def find_cat(string):
    categories = {"culture()": "Culture", "event()": "Event", "meme()": "Meme", 
                  "person()": "Person", "site()": "Site", "subculture()": "Subculture"}
    for pattern in categories.keys():
        result = re.match(pattern,string)
        if result: return(categories[pattern])
    return("Other")

def get_one_value(df,URL,value_col):
    all = df[df["URL"]==URL][value_col]
    return all.iloc[0]

In [119]:
df = pd.read_csv('dags/1641208770_link.csv')
df = df[["URL","Title","Category","Parent","SocialMediaDescription"]]
links = pd.read_csv('dags/1641146515_meme_link.csv')
links = links[["Link","LinkName"]]
links = links.drop_duplicates()
missing_links = links[~links["Link"].isin(df["URL"])]
missing_links.columns = ["URL","Title"]
missing_links["Parent"] = np.NAN
missing_links["SocialMediaDescription"] = np.NAN
missing_links["Category2"] = links.apply(lambda row: str(row["Link"]).split("https://knowyourmeme.com/")[-1].split("memes/")[-1].split("/")[0].strip(),axis=1)
missing_links["Category"] = missing_links.apply(lambda row: find_cat(str(row["Category2"])),axis=1)
missing_links = missing_links[["URL","Title","Category","Parent","SocialMediaDescription"]]
df = df.append(missing_links)
df = df.drop_duplicates()
## find URLs with multiple titles or categories
stat = df.groupby(by="URL").count().sort_values(by="Title",ascending=False)
urls = list(stat[stat["Title"]>1].index) + list(stat[stat["Category"]>1].index)
repeating_urls = df[df["URL"].isin(urls)].sort_values(by="URL")
repeating_urls = repeating_urls[["URL","Parent","SocialMediaDescription"]]
repeating_urls = repeating_urls.drop_duplicates()
## find one title for URLs which had repeating titles
repeating_urls["Title"] = repeating_urls.apply(lambda row: get_one_value(df,str(row["URL"]),"Title"),axis=1)
repeating_urls["Category"] = repeating_urls.apply(lambda row: get_one_value(df,str(row["URL"]),"Category"),axis=1)
repeating_urls = repeating_urls[["URL","Title","Category","Parent","SocialMediaDescription"]]
# remove repeating titles and join fixed rows
df = df[~df["URL"].isin(urls)]
df = df.append(repeating_urls)
df = df.drop_duplicates()
df.groupby(by="URL").count().sort_values(by="Category",ascending=False)


/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1596/3476801643.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_links["Parent"] = np.NAN
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1596/3476801643.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_links["SocialMediaDescription"] = np.NAN
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_1596/3476801643.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,Title,Category,Parent,SocialMediaDescription
URL,,,,
http://--buttslut.tumblr.com/post/92488939150/my-dash-did-a-thing,1,1,0,0
https://knowyourmeme.com/memes/yello-oh-yeah,1,1,0,1
https://knowyourmeme.com/memes/yeet,1,1,0,1
https://knowyourmeme.com/memes/yeees,1,1,0,1
https://knowyourmeme.com/memes/yee,1,1,1,1
...,...,...,...,...
http://www.tumblr.com/tagged/%28%CA%98%E2%80%BF%CA%98%29%E3%83%8E%E2%9C%BF,1,1,0,0
http://www.tumblr.com/tagged/%28%C2%B4%EF%BD%A5%CF%89%EF%BD%A5%60%29,1,1,0,0
http://www.tumblr.com/tagged/!headcanon,1,1,0,0


In [116]:
rep_urls = repeating_urls["URL"]
rep_urls = rep_urls.drop_duplicates()
rep_urls.shape

(796,)

In [110]:
missing_url = missing_links["URL"]
missing_url = missing_url.drop_duplicates()
missing_url.shape

(60860,)

In [100]:
df[df["URL"].isin(df[df["Title"].isna()]["URL"])].groupby(by="URL").count().sort_values(by="Category",ascending=False)

,Title,Category,Parent,SocialMediaDescription
URL,,,,
http://twitter.com/JBtoColorado,1,2,0,0
https://knowyourmeme.com/memes/9gag,1,2,0,0
https://knowyourmeme.com/memes/shrek,1,2,0,0
http://singaporeseen.stomp.com.sg/stomp/sgseen/what_bugs_me/804212/pai_kia_girls_on_rise_in_spore_one_boasts_that.html,1,2,0,0
https://knowyourmeme.com/memes/x-in-g-major,1,2,0,0
...,...,...,...,...
http://s32.photobucket.com/albums/d13/fred26/?action=view&current=2ew1l42.jpg,0,1,0,0
http://s279.photobucket.com/albums/kk134/tucksman/?action=view&current=1274555772395.jpg,0,1,0,0
http://s1238.photobucket.com/albums/ff492/machineking0011/?action=view&current=untitled-3.png,0,1,0,0


In [94]:
df[df["URL"]=='https://knowyourmeme.com/memes/metroid']

,URL,Title,Category,Parent,SocialMediaDescription
7027,https://knowyourmeme.com/memes/metroid,NaN,Other,NaN,NaN
7259,https://knowyourmeme.com/memes/metroid,Metroid,Other,NaN,NaN


In [83]:
stat

,Title,Category,Parent,SocialMediaDescription
URL,,,,
https://knowyourmeme.com/memes/novelty-twitter-accounts,14,14,0,0
https://knowyourmeme.com/memes/subcultures/ironic-memes,13,13,0,0
https://knowyourmeme.com/memes/subcultures/youtube-poop,10,10,0,0
https://knowyourmeme.com/memes/reaction-faces,9,9,0,0
https://knowyourmeme.com/memes/subcultures/youtube-poop-ytp,8,8,0,0
...,...,...,...,...
https://knowyourmeme.com/memes/blessed-image--2,0,1,0,0
https://knowyourmeme.com/memes/ytp-noun-replacement,0,1,0,0
http://img46.imageshack.us/img46/2759/oldshitcopy.png,0,1,0,0


In [ ]:
https://knowyourmeme.com/memes/metroid

In [66]:
df = pd.read_csv('dags/1641138757_link.csv')
about_links = pd.read_csv('dags/1641134743_meme_about_link.csv')
origin_links = pd.read_csv('dags/1641134743_meme_origin_link.csv')
spread_links = pd.read_csv('dags/1641134743_meme_spread_link.csv')
notex_links = pd.read_csv('dags/1641134743_meme_notex_link.csv')
searchint_links = pd.read_csv('dags/1641134743_meme_searchint_link.csv')
extref_links = pd.read_csv('dags/1641134743_meme_extref_link.csv')
all_links = about_links.append(origin_links).append(spread_links).append(notex_links).append(searchint_links).append(extref_links)
all_links = all_links[~all_links["Link"].isin(df["URL"])]
all_links = all_links.drop_duplicates()
missing_links = pd.DataFrame()
missing_links["URL"] = all_links["Link"]
missing_links["Title"] = all_links["LinkName"]
missing_links["Parent"] = np.NAN
missing_links["SocialMediaDescription"] = np.NAN
missing_links["Category2"] = all_links.apply(lambda row: str(row["Link"]).split("https://knowyourmeme.com/")[-1].split("memes/")[-1].split("/")[0].strip(),axis=1)
missing_links["Category"] = missing_links.apply(lambda row: find_cat(str(row["Category2"])),axis=1)
missing_links = missing_links[["URL","Title","Category","Parent","SocialMediaDescription"]]
df = df[["URL","Title","Category","Parent","SocialMediaDescription"]]
df = df.append(missing_links)
df = df.drop_duplicates()

stat = df.groupby(by="URL").count().sort_values(by="Title",ascending=False)
urls = list(stat[stat["Title"]>1].index)

repeating_titles = df[df["URL"].isin(urls)].sort_values(by="URL")
repeating_titles = repeating_titles[["URL","Category","Parent","SocialMediaDescription"]]
repeating_titles = repeating_titles.drop_duplicates()
repeating_titles["Title"] = repeating_titles.apply(lambda row: get_one_title(df,str(row["URL"])),axis=1)
repeating_titles = repeating_titles[["URL","Title","Category","Parent","SocialMediaDescription"]]

df = df[~df["URL"].isin(urls)]
df = df.append(repeating_titles)
df = df.drop_duplicates()
df

,URL,Title,Category,Parent,SocialMediaDescription
0,https://knowyourmeme.com/memes/this-is-relevan...,This is Relevant To My Interests,Meme,NaN,This is relevant to my interests is a popular ...
1,https://knowyourmeme.com/memes/roflcopter,ROFLcopter,Meme,https://knowyourmeme.com/memes/lol-ascii,An ASCII helicopter becomes an Internet meme. ...
2,https://knowyourmeme.com/memes/bitches-dont-know,Bitches Dont Know,Meme,NaN,A braggardly young man flaunts his genitals vi...
3,https://knowyourmeme.com/memes/leave-britney-a...,Leave Britney Alone,Meme,https://knowyourmeme.com/memes/people/chris-cr...,“Leave Britney Alone” is a viral video in whic...
4,https://knowyourmeme.com/memes/o-rly,O RLY?,Meme,https://knowyourmeme.com/memes/lolspeak-chanspeak,O RLY is internet slang for OH REALLY? with im...
...,...,...,...,...,...
25888,https://www.youtube.com/watch?v=clfSc9c4mcA&or...,Collection of failed allahu akbar,Other,NaN,NaN
11975,https://www.youtube.com/watch?v=hmjlA4LSaN4,Everyday Struggle,Other,NaN,NaN
49059,https://www.youtube.com/watch?v=oh87njiWTmw,Scary Maze prank – The Original,Other,NaN,NaN
351,https://www.youtube.com/watch?v=xOJybnDenbI,MOAR KRABS RETURNS,Other,NaN,NaN


In [67]:
df[df["URL"]=="https://knowyourmeme.com/memes/metroid"]

,URL,Title,Category,Parent,SocialMediaDescription
7027,https://knowyourmeme.com/memes/metroid,NaN,Other,NaN,NaN
7259,https://knowyourmeme.com/memes/metroid,Metroid,Other,NaN,NaN


In [58]:
stat = df.groupby(by="URL").count().sort_values(by="Title",ascending=False)
urls = list(stat[stat["Title"]>1].index)

In [59]:
urls

['https://knowyourmeme.com/memes/novelty-twitter-accounts',
 'https://knowyourmeme.com/memes/subcultures/ironic-memes',
 'https://knowyourmeme.com/memes/subcultures/youtube-poop',
 'https://knowyourmeme.com/memes/reaction-faces',
 'https://knowyourmeme.com/memes/subcultures/youtube-poop-ytp',
 'https://knowyourmeme.com/memes/fail',
 'https://knowyourmeme.com/memes/bait-and-switch-videos-pictures',
 'https://knowyourmeme.com/memes/forced-meme',
 'https://knowyourmeme.com/memes/trolling',
 'https://knowyourmeme.com/memes/phonetic-translations',
 'https://knowyourmeme.com/memes/sites/routoftheloop',
 'https://knowyourmeme.com/memes/rage-comics',
 'https://knowyourmeme.com/memes/subcultures/montage-parodies',
 'http://www.foxnews.com/story/0',
 'https://knowyourmeme.com/memes/youtube-poop',
 'https://knowyourmeme.com/memes/subcultures/gif',
 'https://knowyourmeme.com/memes/subcultures/photoshop',
 'https://knowyourmeme.com/memes/internet-slang#IM',
 'http://www.time.com/time/specials/packa

In [52]:
df[~df["URL"].isin(df1["URL"])]

,URL,Title,Category,Parent,SocialMediaDescription


In [49]:
def get_one_title(df,URL):
    all_titles = df[df["URL"]==URL]["Title"]
    #all_titles = all_titles.drop_duplicates()
    #all_titles = all_titles.sort_values()
    return all_titles.iloc[0]

In [50]:
get_one_title(df,'https://knowyourmeme.com/memes/subcultures/ironic-memes')

'ironic memes'

In [7]:
df.shape

(81733, 5)

In [6]:
df1.shape

(77573, 4)

In [46]:
keys = df["URL"]
keys = keys.drop_duplicates()

In [47]:
keys.shape

(77573,)

In [52]:
df[df["URL"]=="https://knowyourmeme.com/memes/subcultures/ironic-memes"]

,URL,Title,Category,Parent,SocialMediaDescription
16,https://knowyourmeme.com/memes/subcultures/iro...,ironic memes,Subculture,NaN,NaN
409,https://knowyourmeme.com/memes/subcultures/iro...,ironic,Subculture,NaN,NaN
1637,https://knowyourmeme.com/memes/subcultures/iro...,ironically,Subculture,NaN,NaN
2262,https://knowyourmeme.com/memes/subcultures/iro...,use the title ironically,Subculture,NaN,NaN
2840,https://knowyourmeme.com/memes/subcultures/iro...,ironic meme,Subculture,NaN,NaN
3403,https://knowyourmeme.com/memes/subcultures/iro...,Ironic Memes,Subculture,NaN,NaN
3851,https://knowyourmeme.com/memes/subcultures/iro...,ironic expression,Subculture,NaN,NaN
5296,https://knowyourmeme.com/memes/subcultures/iro...,ironic following.,Subculture,NaN,NaN
5678,https://knowyourmeme.com/memes/subcultures/iro...,ironic following,Subculture,NaN,NaN
5896,https://knowyourmeme.com/memes/subcultures/iro...,ironic fan,Subculture,NaN,NaN


In [51]:
df.groupby(by="URL").count().sort_values(by="Category",ascending=False)[:20]

,Title,Category,Parent,SocialMediaDescription
URL,,,,
https://knowyourmeme.com/memes/novelty-twitter-accounts,14,14,0,0
https://knowyourmeme.com/memes/subcultures/ironic-memes,13,13,0,0
https://knowyourmeme.com/memes/subcultures/youtube-poop,10,10,0,0
https://knowyourmeme.com/memes/reaction-faces,9,9,0,0
https://knowyourmeme.com/memes/fail,8,8,0,0
https://knowyourmeme.com/memes/subcultures/youtube-poop-ytp,8,8,0,0
http://www.foxnews.com/story/0,7,7,0,0
https://knowyourmeme.com/memes/bait-and-switch-videos-pictures,7,7,0,0
https://knowyourmeme.com/memes/subcultures/montage-parodies,7,7,0,0


In [27]:
df.shape

(81794, 5)

In [28]:
df.count()

URL                       81794
Title                     81655
Category                  81794
Parent                     6897
SocialMediaDescription    16693
dtype: int64

In [29]:
df.groupby(by="Category").count().sort_values(by="URL")

,URL,Title,Parent,SocialMediaDescription
Category,,,,
Culture,642,642,50,156
Site,1028,1027,130,379
Person,1175,1175,324,1173
Event,1213,1213,524,1114
Subculture,2244,2243,540,1233
Meme,12666,12666,5329,12638
Other,62826,62689,0,0


In [30]:
about_links[~about_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


In [31]:
origin_links[~origin_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


In [32]:
spread_links[~spread_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


In [33]:
notex_links[~notex_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


In [34]:
searchint_links[~searchint_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


In [35]:
extref_links[~extref_links["Link"].isin(df["URL"])]

,URL,LinkName,Link


#### Enrichment idea: collect nbr of views for each meme - too time expensive

In [3]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests

url = 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests'
req = Request(url)
html_page = urlopen(req)
soup = BeautifulSoup(html_page, 'lxml')

In [4]:
soup.find_all("div", "views")

[<div class="views" title="136,250 Views"></div>]

In [5]:
int(str(soup.find_all("div", "views")).split('title="')[1].split('Views')[0].replace(",","").strip())

136250

In [6]:
def nbr_of_views(url):
    if requests.get(url).status_code != 104:
        req = Request(url)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page,'html.parser')
        views = int(str(soup.find_all("div", "views")).split('title="')[1].split('Views')[0].replace(",","").strip())
    else: views=0
    return views

In [8]:
import pandas as pd

df = pd.read_csv('dags/1640802608_filtered.csv')

In [9]:
import time

In [ ]:
for i in range(df.shape[0]):
    start = time.time()
    print(df.iloc[i,1],nbr_of_views(df.iloc[i,1]))
    print(time.time()-start)
    if i==20: break

https://knowyourmeme.com/memes/this-is-relevant-to-my-interests 136250
4.1860010623931885
https://knowyourmeme.com/memes/roflcopter 538418
3.963809013366699
https://knowyourmeme.com/memes/bitches-dont-know 200016
7.369422197341919
https://knowyourmeme.com/memes/leave-britney-alone 211469
3.494152307510376
https://knowyourmeme.com/memes/o-rly 1070570
4.821468830108643
https://knowyourmeme.com/memes/yatta 41239
2.934696912765503
https://knowyourmeme.com/memes/in-soviet-russia 920032
3.2886509895324707
https://knowyourmeme.com/memes/domo 330654
3.5967278480529785
https://knowyourmeme.com/memes/i-like-turtles 600184
3.710955858230591
https://knowyourmeme.com/memes/diabeetus 637278
4.396682024002075
https://knowyourmeme.com/memes/star-wars-kid 255776
4.700733184814453
https://knowyourmeme.com/memes/a-winner-is-you 406019
3.503669023513794
https://knowyourmeme.com/memes/gi-joe-psas 205914
4.555746078491211
https://knowyourmeme.com/memes/recession-raven 6345
3.213191270828247
https://knowyour

### Make separate tables:

#### Pattern search

In [426]:
import re 

def matched_pattern(pattern,string):
    result = re.match(pattern,string)
    if result: return(pattern)
    else: return("")
    
def matched_string(pattern,string):
    result = re.match(pattern,string)
    if result: return(string)
    else: return("")

def opposite_match(pattern,string):
    result = re.match(pattern,string)
    if result!=None: return
    else: return(string)

def pat_pos(pattern,string):
    match = re.search(pattern,string)
    if match: return match.start()
    else: return -1
    
pattern = 'https(...)'
string1 = 'https://knowyourmeme.com/memes/sites/youtube'
string2 = 'youtube'
print(matched_string(pattern,string1))
print(opposite_match(pattern,string2))

https://knowyourmeme.com/memes/sites/youtube
youtube


#### df -- all filtered data

In [ ]:
import pandas as pd

df = pd.read_csv('dags/1640802608_filtered.csv')

In [89]:
examples = df.sample(50)
examples.to_excel("examples.xlsx")

In [430]:
df.shape

(12621, 33)

In [158]:
df.columns

Index(['Title', 'URL', 'TimeUpdated', 'Image', 'TimeAdded', 'Tags', 'Keywords',
       'Parent', 'References', 'ImageWidth', 'ImageHeight',
       'SocialMediaDescription', 'Status', 'Origin', 'Year', 'Type',
       'AboutText', 'AboutImages', 'AboutLinks', 'OriginText', 'OriginImages',
       'OriginLinks', 'SpreadText', 'SpreadImages', 'SpreadLinks',
       'NotExamplesText', 'NotExamplesImages', 'NotExamplesLinks',
       'SearchIntText', 'SearchIntImages', 'SearchIntLinks', 'ExtRefText',
       'ExtRefLinks'],
      dtype='object')

In [10]:
df.head(2)

,Title,URL,TimeUpdated,Image,TimeAdded,Tags,Keywords,Parent,References,ImageWidth,...,SpreadImages,SpreadLinks,NotExamplesText,NotExamplesImages,NotExamplesLinks,SearchIntText,SearchIntImages,SearchIntLinks,ExtRefText,ExtRefLinks
0,This is Relevant To My Interests,https://knowyourmeme.com/memes/this-is-relevan...,2019-01-09 03:01:38,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:12:41,"[image macros, comment, comments, approval, im...",[This is relevant to my interests],NaN,{},600,...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[LOLcat, https://knowyourmeme.com/memes/lolca...",NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,[Search query volume for the keywords this is ...,NaN,"[[Caturday, https://knowyourmeme.com/memes/cat...",[[1] I Can Has Cheezburger – This is relevant ...,"[[This is relevant to my interests, http://ica..."
1,ROFLcopter,https://knowyourmeme.com/memes/roflcopter,2020-06-05 23:38:57,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:13:27,"[ascii, animated gif, rofl, acronym, initialism]",[roflcopter],https://knowyourmeme.com/memes/lol-ascii,{Encyclopedia Dramatica: https://encyclopediad...,600,...,[https://i.kym-cdn.com/photos/images/newsfeed/...,"[[single-serving site, https://knowyourmeme.co...",NaN,NaN,NaN,"[By September of 2005, searches for the term s...",NaN,NaN,"[[1] Urban Dictionary – roflcopter, [2] Wiktio...","[[roflcopter, https://www.urbandictionary.com/..."


#### Reference

In [409]:
Refs = df[["URL","References"]]
Refs["References"] = Refs.apply(lambda row: str(row["References"]).replace("{","").replace("}","").replace(",:",":"),axis=1)
refers = Refs.apply(lambda row: str(row["References"]).split(", "),axis=1).explode()
refers = pd.DataFrame(refers,columns=["Ref"])
refers["RefLink"] = refers.apply(lambda row: str(row["Ref"])[pat_pos('http(...)',str(row["Ref"])):],axis=1)
refers["RefName"] = refers.apply(lambda row: str(row["Ref"]).split(": http")[0].split(":")[0],axis=1)
refers["RefLink"] = refers.apply(lambda row: str(row["RefLink"]).strip(),axis=1)
refers["RefName"] = refers.apply(lambda row: str(row["RefName"]).strip(),axis=1)
refers["RefLink"] = refers.apply(lambda row: matched_string('https(...)',str(row["RefLink"])),axis=1)
refers = refers[refers["RefLink"]!=""]
Refers = Refs.join(refers)
Refers = Refers.drop(labels=Refers[Refers["RefLink"].isna()].index)
Refers = Refers[["URL","RefName","RefLink"]]
Refers


/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/535293620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Refs["References"] = Refs.apply(lambda row: str(row["References"]).replace("{","").replace("}","").replace(",:",":"),axis=1)


,URL,RefName,RefLink
1,https://knowyourmeme.com/memes/roflcopter,Encyclopedia Dramatica,https://encyclopediadramatica.wiki/index.php/R...
2,https://knowyourmeme.com/memes/bitches-dont-know,Facebook,https://www.facebook.com/search/top/?q=bitches...
2,https://knowyourmeme.com/memes/bitches-dont-know,Reddit,https://www.reddit.com/search?q=bitches+don%27...
2,https://knowyourmeme.com/memes/bitches-dont-know,Twitter,https://twitter.com/search?q=bitches%20don%27t...
2,https://knowyourmeme.com/memes/bitches-dont-know,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
...,...,...,...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Wikipedia,https://en.wikipedia.org/wiki/List_of_Calvin_a...
12617,https://knowyourmeme.com/memes/liberal-douche-...,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
12618,https://knowyourmeme.com/memes/polybius,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
12618,https://knowyourmeme.com/memes/polybius,Wikipedia,https://en.wikipedia.org/wiki/Polybius_%28vide...


In [395]:
Refers.groupby(by="RefName").count().sort_values(by="URL",ascending=False)

,URL,RefLink
RefName,,
,8417,8417
Wikipedia,2017,2017
Urban Dictionary,1466,1466
Meme Generator,858,858
Encyclopedia Dramatica,858,858
...,...,...
Initial D Wiki,1,1
IMDB,1,1
Humoroso,1,1


In [396]:
Refers.groupby(by="URL").count().sort_values(by="RefName",ascending=False)

,RefName,RefLink
URL,,
https://knowyourmeme.com/memes/rickroll,8,8
https://knowyourmeme.com/memes/doge,8,8
https://knowyourmeme.com/memes/pepe-the-frog,8,8
https://knowyourmeme.com/memes/chuck-norris-facts,7,7
https://knowyourmeme.com/memes/hide-the-pain-harold,7,7
...,...,...
https://knowyourmeme.com/memes/help-desk-jesus,1,1
https://knowyourmeme.com/memes/help-i-accidentally-build-a-shelf,1,1
https://knowyourmeme.com/memes/help-im-trapped-in-a-factory,1,1


In [398]:
stats = Refers.groupby(by="RefName").count()
stats.to_excel("Ref_stat.xlsx")

In [157]:
Refers[Refers["RefName"]==""].index
Refers = Refers.drop(labels=Refers[Refers["RefName"]==""].index)
Refers

,URL,RefName,RefLink
1,https://knowyourmeme.com/memes/roflcopter,Encyclopedia Dramatica,https://encyclopediadramatica.wiki/index.php/R...
2,https://knowyourmeme.com/memes/bitches-dont-know,Facebook,https://www.facebook.com/search/top/?q=bitches...
2,https://knowyourmeme.com/memes/bitches-dont-know,Reddit,https://www.reddit.com/search?q=bitches+don%27...
2,https://knowyourmeme.com/memes/bitches-dont-know,Twitter,https://twitter.com/search?q=bitches%20don%27t...
2,https://knowyourmeme.com/memes/bitches-dont-know,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
...,...,...,...
12617,https://knowyourmeme.com/memes/liberal-douche-...,Meme Generator,http://memegenerator.net/Liberal-Douche-Garofalo
12617,https://knowyourmeme.com/memes/liberal-douche-...,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
12618,https://knowyourmeme.com/memes/polybius,Urban Dictionary,https://www.urbandictionary.com/define.php?ter...
12618,https://knowyourmeme.com/memes/polybius,Wikipedia,https://en.wikipedia.org/wiki/Polybius_%28vide...


#### Meme

In [370]:
Memes = df[['URL','Title','TimeUpdated', 'Image', 'TimeAdded', 
            'Keywords', 'Parent', 'SocialMediaDescription', 'Status', 'Origin', 'Year']]
Memes["Keywords"] = Memes.apply(lambda row: str(row["Keywords"]).replace("[","").replace("]","").strip(),axis=1)
Memes = Memes.drop_duplicates()
Memes

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/139846423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Memes["Keywords"] = Memes.apply(lambda row: str(row["Keywords"]).replace("[","").replace("]","").strip(),axis=1)


,URL,Title,TimeUpdated,Image,TimeAdded,Keywords,Parent,SocialMediaDescription,Status,Origin,Year
0,https://knowyourmeme.com/memes/this-is-relevan...,This is Relevant To My Interests,2019-01-09 03:01:38,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:12:41,This is relevant to my interests,NaN,This is relevant to my interests is a popular ...,confirmed,I Can Has Cheezburger,2006
1,https://knowyourmeme.com/memes/roflcopter,ROFLcopter,2020-06-05 23:38:57,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:13:27,roflcopter,https://knowyourmeme.com/memes/lol-ascii,An ASCII helicopter becomes an Internet meme. ...,confirmed,Warcraft III,2004
2,https://knowyourmeme.com/memes/bitches-dont-know,Bitches Dont Know,2020-12-14 23:03:19,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:13:01,bitches dont know,NaN,A braggardly young man flaunts his genitals vi...,confirmed,MySpace,2006
3,https://knowyourmeme.com/memes/leave-britney-a...,Leave Britney Alone,2021-02-12 11:28:59,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:11:00,"leave britney alone, chris crocker",https://knowyourmeme.com/memes/people/chris-cr...,“Leave Britney Alone” is a viral video in whic...,confirmed,YouTube,2007
4,https://knowyourmeme.com/memes/o-rly,O RLY?,2021-03-13 13:58:19,https://i.kym-cdn.com/entries/icons/original/0...,2008-12-12 20:10:42,o rly,https://knowyourmeme.com/memes/lolspeak-chanspeak,O RLY is internet slang for OH REALLY? with im...,confirmed,Something Awful,2003
...,...,...,...,...,...,...,...,...,...,...,...
12616,https://knowyourmeme.com/memes/cee-los-muppet-...,Cee Los Muppet Costume,2019-12-16 22:37:06,https://i.kym-cdn.com/entries/icons/original/0...,2011-02-14 04:20:13,Cee Lo Bird,NaN,R&B singer / songwriter Cee Lo is the talk of ...,submission,Tumblr,2011
12617,https://knowyourmeme.com/memes/liberal-douche-...,Liberal Douche Garofalo,2021-04-04 10:22:41,https://i.kym-cdn.com/entries/icons/original/0...,2012-03-28 16:43:36,Liberal Douche Garofalo,https://knowyourmeme.com/memes/advice-animals,Liberal Douche Garofalo is an advice animal im...,confirmed,4chan,2009
12618,https://knowyourmeme.com/memes/polybius,Polybius,2021-03-29 20:54:44,https://i.kym-cdn.com/entries/icons/original/0...,2011-08-26 16:36:19,polybius game,https://knowyourmeme.com/memes/creepypasta,Polybius is a fictional arcade game that was s...,confirmed,Poly-Play,1981
12619,https://knowyourmeme.com/memes/donald-trump-re...,Donald Trump Reacts to Ted Cruz,2019-11-07 13:48:39,https://i.kym-cdn.com/entries/icons/original/0...,2016-07-21 16:52:51,nan,https://knowyourmeme.com/memes/people/donald-t...,Ted Cruz refused to endorse Donald Trump at th...,submission,Reddit,2016


In [371]:
Memes.isna().sum()

URL                          0
Title                        0
TimeUpdated                  0
Image                        0
TimeAdded                    0
Keywords                     0
Parent                    7311
SocialMediaDescription      16
Status                       0
Origin                       3
Year                         0
dtype: int64

### All Links

#### About

In [298]:
About = df[["URL","AboutLinks"]]
About["AboutLinks"] = About.apply(lambda row: str(row["AboutLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
About = About[About["AboutLinks"]!='nan']
links = About.apply(lambda row: str(row["AboutLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
About = About.join(links)
About = About.drop(labels=About[About["Links"].isna()].index)
About = About[["URL","LinkName","Link"]]
About = About.drop_duplicates()
About

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/130885957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  About["AboutLinks"] = About.apply(lambda row: str(row["AboutLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,image macros,https://knowyourmeme.com/memes/image-macros
0,https://knowyourmeme.com/memes/this-is-relevan...,animals,https://knowyourmeme.com/memes/cultures/animals
1,https://knowyourmeme.com/memes/roflcopter,Lollerskates,https://knowyourmeme.com/memes/lollerskates
1,https://knowyourmeme.com/memes/roflcopter,GIF,https://knowyourmeme.com/memes/cultures/gif
1,https://knowyourmeme.com/memes/roflcopter,ASCII,https://knowyourmeme.com/memes/subcultures/asc...
...,...,...,...
12619,https://knowyourmeme.com/memes/donald-trump-re...,Donald Trump,https://knowyourmeme.com/memes/people/donald-t...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,MAD,https://knowyourmeme.com/memes/subcultures/mads
12620,https://knowyourmeme.com/memes/chocobo-racing-...,Nico Nico Douga,https://knowyourmeme.com/memes/sites/nico-nico...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,YouTube Poop Music Videos,https://knowyourmeme.com/memes/ytpmvs/


In [210]:
About.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
image macro,485,485
YouTube,466,466
catchphrase,387,387
Twitter,294,294
4chan,285,285
...,...,...
Nokia N-Gage.,1,1
North Korean,1,1
North Koreas,1,1


#### Spread

In [299]:
Spread = df[["URL","SpreadLinks"]]
Spread["SpreadLinks"] = Spread.apply(lambda row: str(row["SpreadLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
Spread = Spread[Spread["SpreadLinks"]!='nan']
links = Spread.apply(lambda row: str(row["SpreadLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
Spread = Spread.join(links)
Spread = Spread.drop(labels=Spread[Spread["Links"].isna()].index)
Spread = Spread[["URL","LinkName","Link"]]
Spread = Spread.drop_duplicates()
Spread

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/2980155167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spread["SpreadLinks"] = Spread.apply(lambda row: str(row["SpreadLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,LOLcat,https://knowyourmeme.com/memes/lolcats
0,https://knowyourmeme.com/memes/this-is-relevan...,Internet,https://knowyourmeme.com/memes/cultures/the-in...
0,https://knowyourmeme.com/memes/this-is-relevan...,cat,https://knowyourmeme.com/memes/cultures/cats
0,https://knowyourmeme.com/memes/this-is-relevan...,Urban Dictionary,https://knowyourmeme.com/memes/sites/urban-dic...
0,https://knowyourmeme.com/memes/this-is-relevan...,Cheezburger,https://knowyourmeme.com/memes/sites/cheezburger
...,...,...,...
12618,https://knowyourmeme.com/memes/polybius,Cracked,https://knowyourmeme.com/memes/sites/cracked
12618,https://knowyourmeme.com/memes/polybius,Yahoo,https://knowyourmeme.com/memes/sites/yahoo
12618,https://knowyourmeme.com/memes/polybius,Internet,https://knowyourmeme.com/memes/cultures/the-in...
12618,https://knowyourmeme.com/memes/polybius,The Simpsons,https://knowyourmeme.com/memes/subcultures/the...


In [296]:
Spread.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
Facebook,1001,1001
Tumblr,991,991
Redditor,934,934
Twitter,745,745
Urban Dictionary,579,579
...,...,...
K Pop,1,1
Justin Whang,1,1
Justin Biebers,1,1


#### Origin

In [300]:
Origin = df[["URL","OriginLinks"]]
Origin["OriginLinks"] = Origin.apply(lambda row: str(row["OriginLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
Origin = Origin[Origin["OriginLinks"]!='nan']
links = Origin.apply(lambda row: str(row["OriginLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
Origin = Origin.join(links)
Origin = Origin.drop(labels=Origin[Origin["Links"].isna()].index)
Origin = Origin[["URL","LinkName","Link"]]
Origin = Origin.drop_duplicates()
Origin

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/2572518925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Origin["OriginLinks"] = Origin.apply(lambda row: str(row["OriginLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,4chan,https://knowyourmeme.com/memes/sites/4chan
0,https://knowyourmeme.com/memes/this-is-relevan...,gaming,https://knowyourmeme.com/memes/cultures/gaming
0,https://knowyourmeme.com/memes/this-is-relevan...,NeoGAF,https://knowyourmeme.com/memes/sites/neogaf
1,https://knowyourmeme.com/memes/roflcopter,Urban Dictionary,https://knowyourmeme.com/memes/sites/urban-dic...
1,https://knowyourmeme.com/memes/roflcopter,gyrocopter,https://knowyourmeme.com/photos/296242-roflcopter
...,...,...,...
12617,https://knowyourmeme.com/memes/liberal-douche-...,4chan,https://knowyourmeme.com/memes/sites/4chan
12617,https://knowyourmeme.com/memes/liberal-douche-...,/b/,https://knowyourmeme.com/memes/sites/b
12617,https://knowyourmeme.com/memes/liberal-douche-...,College Liberal,https://knowyourmeme.com/memes/college-liberal
12619,https://knowyourmeme.com/memes/donald-trump-re...,Republican National Convention,https://knowyourmeme.com/memes/events/2016-rep...


In [294]:
Origin.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
YouTube,484,484
4chan,339,339
Twitter,309,309
Tumblr,307,307
Facebook,188,188
...,...,...
Karen,1,1
Katamari Damacy,1,1
Katy Perry,1,1


#### Notable Examples

In [291]:
NotEx = df[["URL","NotExamplesLinks"]]
NotEx["NotExamplesLinks"] = NotEx.apply(lambda row: str(row["NotExamplesLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
NotEx = NotEx[NotEx["NotExamplesLinks"]!='nan']
links = NotEx.apply(lambda row: str(row["NotExamplesLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
NotEx = NotEx.join(links)
NotEx = NotEx.drop(labels=NotEx[NotEx["Links"].isna()].index)
NotEx = NotEx[["URL","LinkName","Link"]]
NotEx = NotEx.drop_duplicates()
NotEx

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/1254863117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NotEx["NotExamplesLinks"] = NotEx.apply(lambda row: str(row["NotExamplesLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
141,https://knowyourmeme.com/memes/op-is-a-faggot,Tumblr,https://knowyourmeme.com/memes/sites/tumblr
163,https://knowyourmeme.com/memes/can-can,Enemy Controller,https://knowyourmeme.com/memes/enemy-controller
163,https://knowyourmeme.com/memes/can-can,MikuMikuDance,https://knowyourmeme.com/memes/miku-miku-dance
220,https://knowyourmeme.com/memes/successful-blac...,Quickmeme,https://knowyourmeme.com/memes/sites/quickmeme
269,https://knowyourmeme.com/memes/magikarp,Fan art,https://knowyourmeme.com/memes/cultures/fan-art
...,...,...,...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Batman,https://knowyourmeme.com/memes/subcultures/batman
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Breaking Bad,https://knowyourmeme.com/memes/subcultures/bre...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Walking Dead,https://knowyourmeme.com/memes/subcultures/the...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Pokemon,https://knowyourmeme.com/memes/subcultures/pok...


In [292]:
NotEx.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
Tumblr,39,39
Quickmeme,18,18
,15,15
2014,14,14
YouTube,11,11
...,...,...
Lawyer Dog,1,1
Leaked Senkaku Video,1,1
Leonardo DiCaprios,1,1


#### Search Interest

In [289]:
SearchInt = df[["URL","SearchIntLinks"]]
SearchInt["SearchIntLinks"] = SearchInt.apply(lambda row: str(row["SearchIntLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
SearchInt = SearchInt[SearchInt["SearchIntLinks"]!='nan']
links = SearchInt.apply(lambda row: str(row["SearchIntLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
SearchInt = SearchInt.join(links)
SearchInt = SearchInt.drop(labels=SearchInt[SearchInt["Links"].isna()].index)
SearchInt = SearchInt[["URL","LinkName","Link"]]
SearchInt = SearchInt.drop_duplicates()
SearchInt


/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/856783126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SearchInt["SearchIntLinks"] = SearchInt.apply(lambda row: str(row["SearchIntLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,Caturday,https://knowyourmeme.com/memes/caturday
15,https://knowyourmeme.com/memes/one-does-not-si...,gif,https://knowyourmeme.com/memes/cultures/gif
15,https://knowyourmeme.com/memes/one-does-not-si...,Rick Santorum,https://knowyourmeme.com/memes/people/rick-san...
240,https://knowyourmeme.com/memes/advice-animals,Courage Wolf,https://knowyourmeme.com/memes/courage-wolf
240,https://knowyourmeme.com/memes/advice-animals,Google,https://knowyourmeme.com/memes/sites/google
...,...,...,...
12495,https://knowyourmeme.com/memes/the-burger-king,BK Joe,https://knowyourmeme.com/videos/18221-the-burg...
12524,https://knowyourmeme.com/memes/hello-my-future...,Google,https://knowyourmeme.com/memes/sites/google
12576,https://knowyourmeme.com/memes/i-herd-u-liek-m...,April Fools Day,https://knowyourmeme.com/memes/events/april-fo...
12577,https://knowyourmeme.com/memes/crank-that-soul...,Collipark Records,https://en.wikipedia.org/wiki/Mr._Collipark


In [290]:
SearchInt.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
Google,36,36
Wikipedia,7,7
YouTube,5,5
YT.com,3,3
Facebook,3,3
...,...,...
False Flag,1,1
Dipity Internet Meme Timeline,1,1
Courage Wolf,1,1


#### External Reference

In [287]:
ExtRef = df[["URL","ExtRefLinks"]]
ExtRef["ExtRefLinks"] = ExtRef.apply(lambda row: str(row["ExtRefLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
ExtRef = ExtRef[ExtRef["ExtRefLinks"]!='nan']
links = ExtRef.apply(lambda row: str(row["ExtRefLinks"]).split(","),axis=1).explode()
links = pd.DataFrame(links,columns=["Links"])
links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
links = links[links["Link"] != ""]
ExtRef = ExtRef.join(links)
ExtRef = ExtRef.drop(labels=ExtRef[ExtRef["Links"].isna()].index)
ExtRef = ExtRef[["URL","LinkName","Link"]]
ExtRef = ExtRef.drop_duplicates()
ExtRef

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/579960423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ExtRef["ExtRefLinks"] = ExtRef.apply(lambda row: str(row["ExtRefLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,This is relevant to my interests,http://icanhascheezburger.com/2007/01/11/this-...
0,https://knowyourmeme.com/memes/this-is-relevan...,Rumble Roses spawns its own JAV,http://www.neogaf.com/forum/showthread.php?t=6...
0,https://knowyourmeme.com/memes/this-is-relevan...,buro9,http://www.flickr.com/photos/buro9/
0,https://knowyourmeme.com/memes/this-is-relevan...,Relevant to my interests,http://encyclopediadramatica.rs/Relevant_to_my...
0,https://knowyourmeme.com/memes/this-is-relevan...,Encyclopedia Dramatica Revision History,http://web.archive.org/web/20100225095835/http...
...,...,...,...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,シドのテストコース,http://www.nicovideo.jp/tag/%E3%82%B7%E3%83%89...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,チョコボレーシング(49歳),http://www.nicovideo.jp/watch/sm12083872
12620,https://knowyourmeme.com/memes/chocobo-racing-...,パンツじゃないもんレーシング,http://www.nicovideo.jp/watch/sm12322218
12620,https://knowyourmeme.com/memes/chocobo-racing-...,チョコボレエﾞエーイシングu3000,http://www.nicovideo.jp/watch/sm9346706


In [288]:
ExtRef.groupby(by="LinkName").count().sort_values(by="Link",ascending=False)

,URL,Link
LinkName,,
Home,254,254
Search,147,147
Wayback Machine,69,69
me irl,51,51
search,34,34
...,...,...
Hopefully well get some Fatlus in here,1,1
Hoppip,1,1
Hoppip C&D,1,1


#### All links (About, Spread, Origin, Notable Examples, Serach Interest) with one function

In [310]:
def explode_links(LinksColumn): # df[["URL","NotExamplesLinks"]]
    Table = df[["URL",LinksColumn]]
    Table.columns = ["URL","TabelLinks"]
    Table["TabelLinks"] = Table.apply(lambda row: str(row["TabelLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)
    Table = Table[Table["TabelLinks"]!='nan']
    links = Table.apply(lambda row: str(row["TabelLinks"]).split(","),axis=1).explode()
    links = pd.DataFrame(links,columns=["Links"])
    links["LinkName"] = links.apply(lambda row: str(row["Links"]).split(": ")[0].strip(),axis=1)
    links["Link"] = links.apply(lambda row: str(row["Links"]).split(": ")[-1].strip(),axis=1)
    links["Link"] = links.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
    links = links[links["Link"] != ""]
    Table = Table.join(links)
    Table = Table.drop(labels=Table[Table["Links"].isna()].index)
    Table = Table[["URL","LinkName","Link"]]
    Table = Table.drop_duplicates()
    return Table

In [311]:
explode_links("NotExamplesLinks")

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/997270454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table["TabelLinks"] = Table.apply(lambda row: str(row["TabelLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
141,https://knowyourmeme.com/memes/op-is-a-faggot,Tumblr,https://knowyourmeme.com/memes/sites/tumblr
163,https://knowyourmeme.com/memes/can-can,Enemy Controller,https://knowyourmeme.com/memes/enemy-controller
163,https://knowyourmeme.com/memes/can-can,MikuMikuDance,https://knowyourmeme.com/memes/miku-miku-dance
220,https://knowyourmeme.com/memes/successful-blac...,Quickmeme,https://knowyourmeme.com/memes/sites/quickmeme
269,https://knowyourmeme.com/memes/magikarp,Fan art,https://knowyourmeme.com/memes/cultures/fan-art
...,...,...,...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Batman,https://knowyourmeme.com/memes/subcultures/batman
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Breaking Bad,https://knowyourmeme.com/memes/subcultures/bre...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Walking Dead,https://knowyourmeme.com/memes/subcultures/the...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,Pokemon,https://knowyourmeme.com/memes/subcultures/pok...


In [313]:
explode_links("AboutLinks")

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/997270454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table["TabelLinks"] = Table.apply(lambda row: str(row["TabelLinks"]).replace("[","").replace("]","").replace(", http",": http"),axis=1)


,URL,LinkName,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,image macros,https://knowyourmeme.com/memes/image-macros
0,https://knowyourmeme.com/memes/this-is-relevan...,animals,https://knowyourmeme.com/memes/cultures/animals
1,https://knowyourmeme.com/memes/roflcopter,Lollerskates,https://knowyourmeme.com/memes/lollerskates
1,https://knowyourmeme.com/memes/roflcopter,GIF,https://knowyourmeme.com/memes/cultures/gif
1,https://knowyourmeme.com/memes/roflcopter,ASCII,https://knowyourmeme.com/memes/subcultures/asc...
...,...,...,...
12619,https://knowyourmeme.com/memes/donald-trump-re...,Donald Trump,https://knowyourmeme.com/memes/people/donald-t...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,MAD,https://knowyourmeme.com/memes/subcultures/mads
12620,https://knowyourmeme.com/memes/chocobo-racing-...,Nico Nico Douga,https://knowyourmeme.com/memes/sites/nico-nico...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,YouTube Poop Music Videos,https://knowyourmeme.com/memes/ytpmvs/


### Meme text

In [319]:
MemeText = df[["URL","AboutText","OriginText","SpreadText","NotExamplesText","SearchIntText", "ExtRefText"]]
MemeText["AboutText"] = MemeText.apply(lambda row: str(row["AboutText"]).replace("[","").replace("]","").strip(),axis=1)
MemeText["OriginText"] = MemeText.apply(lambda row: str(row["OriginText"]).replace("[","").replace("]","").strip(),axis=1)
MemeText["SpreadText"] = MemeText.apply(lambda row: str(row["SpreadText"]).replace("[","").replace("]","").strip(),axis=1)
MemeText["NotExamplesText"] = MemeText.apply(lambda row: str(row["NotExamplesText"]).replace("[","").replace("]","").strip(),axis=1)
MemeText["SearchIntText"] = MemeText.apply(lambda row: str(row["SearchIntText"]).replace("[","").replace("]","").strip(),axis=1)
MemeText["ExtRefText"] = MemeText.apply(lambda row: str(row["ExtRefText"]).replace("[[","[").replace("]]","]").strip(),axis=1)
MemeText

/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/3838061993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MemeText["AboutText"] = MemeText.apply(lambda row: str(row["AboutText"]).replace("[","").replace("]","").strip(),axis=1)
/var/folders/hk/kbn1q_sj4jz4lh0m7x05vff80000gn/T/ipykernel_26307/3838061993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MemeText["OriginText"] = MemeText.apply(lambda row: str(row["OriginText"]).replace("[","").replace("]","").strip(),axis=1)
/var/folders

,URL,AboutText,OriginText,SpreadText,NotExamplesText,SearchIntText,ExtRefText
0,https://knowyourmeme.com/memes/this-is-relevan...,This is Relevant To My Interests is an express...,While the phrase is rumored to have originated...,"On November 16th, 2006, Flickr3 user the boy o...",nan,Search query volume for the keywords this is r...,[1] I Can Has Cheezburger – This is relevant t...
1,https://knowyourmeme.com/memes/roflcopter,ROFLcopter is an Internet slang term used as a...,According to an Urban Dictionary1 submission b...,"On March 30th, 2004, the single-serving site R...",nan,"By September of 2005, searches for the term se...","[1] Urban Dictionary – roflcopter, [2] Wiktion..."
2,https://knowyourmeme.com/memes/bitches-dont-know,Bitches Dont Know is an exploitable image macr...,The picture was originally a Myspace photo of ...,One of the earliest archived instances of bitc...,nan,Bitches Dont Know saw a viral takeoff in late ...,[1] Myspace (via Wayback Machine) – Rich Owens...
3,https://knowyourmeme.com/memes/leave-britney-a...,Leave Britney Alone is a viral video in which ...,"On September 10, 2007, Chris Crocker uploaded ...","From the first day, September 10, 2007, Leave ...",In the days following Leave Britney Alone a nu...,nan,"[1] YouTube – Leave Britney Alone, [2] Wikiped..."
4,https://knowyourmeme.com/memes/o-rly,O RLY is internet slang for “OH REALLY?” with ...,The original “O RLY?” Snowy Owl image macro us...,An isolated instance of O RLY image macro feat...,nan,Search queries for O rly peaked in late 2005 t...,[1] AnimalPictureArchives – alt.binaries.pictu...
...,...,...,...,...,...,...,...
12616,https://knowyourmeme.com/memes/cee-los-muppet-...,Cee Lo Bird refers to the colorful costume wor...,"At the annual Grammys Awards on February 13th,...",nan,nan,nan,"[1] Wikipedia – Cee Lo Green, [2] Wikipedia – ..."
12617,https://knowyourmeme.com/memes/liberal-douche-...,Liberal Douche Garofalo is an advice animal st...,Janeanne Garofalo1 is a stand-up comedian and ...,The Memegenerator page3 for Liberal Douche Gar...,nan,Google search queries for Liberal Douche Garof...,"[1] Wikipedia – Janeane Garofalo, [2] Quickmem..."
12618,https://knowyourmeme.com/memes/polybius,Polybius is a fictional arcade game that was s...,The earliest known reference article about Pol...,The first archived inquiry about the game was ...,nan,nan,"[1] Coinop.com – Polybius, [2] Coinop Polybius..."
12619,https://knowyourmeme.com/memes/donald-trump-re...,Donald Trump Reacts to Ted Cruz is a photoshop...,"On July 20th, 2016, United States Senator and ...","The same day, Redditor belisaurius submitted M...",nan,nan,[1] New York Times – Ted Cruz Stirs Convention...


### Meme images into one table

In [321]:
df[['URL', 'AboutImages', 'OriginImages', 'SpreadImages', 'NotExamplesImages', 'SearchIntImages']]

,URL,AboutImages,OriginImages,SpreadImages,NotExamplesImages,SearchIntImages
0,https://knowyourmeme.com/memes/this-is-relevan...,NaN,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN
1,https://knowyourmeme.com/memes/roflcopter,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,NaN
2,https://knowyourmeme.com/memes/bitches-dont-know,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN
3,https://knowyourmeme.com/memes/leave-britney-a...,NaN,NaN,NaN,NaN,NaN
4,https://knowyourmeme.com/memes/o-rly,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/masonry/0...,NaN,NaN
...,...,...,...,...,...,...
12616,https://knowyourmeme.com/memes/cee-los-muppet-...,NaN,NaN,NaN,NaN,NaN
12617,https://knowyourmeme.com/memes/liberal-douche-...,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/masonry/0...,NaN
12618,https://knowyourmeme.com/memes/polybius,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/masonry/0...,NaN,NaN
12619,https://knowyourmeme.com/memes/donald-trump-re...,NaN,[https://i.kym-cdn.com/photos/images/newsfeed/...,[https://i.kym-cdn.com/photos/images/newsfeed/...,NaN,NaN


In [337]:
Images = df[["URL","AboutImages"]]
imglinks = Images.apply(lambda row: str(row["AboutImages"]).replace("[","").replace("[","").split(","),axis=1).explode()
imglinks = pd.DataFrame(imglinks,columns=["Link"])
imglinks[imglinks["Link"]!='nan']
imglinks["Link"] = imglinks.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
imglinks = imglinks[imglinks["Link"] != ""]
Images = Images.join(imglinks)
Images = Images.drop(labels=Images[Images["Link"].isna()].index)
Images = Images[["URL","Link"]]
Images

,URL,Link
11,https://knowyourmeme.com/memes/a-winner-is-you,https://i.kym-cdn.com/photos/images/newsfeed/0...
41,https://knowyourmeme.com/memes/do-it-faggot,https://i.kym-cdn.com/photos/images/newsfeed/0...
49,https://knowyourmeme.com/memes/cereal-guy,https://i.kym-cdn.com/photos/images/newsfeed/0...
56,https://knowyourmeme.com/memes/dr-hax,http://chart.apis.google.com/chart?cht=lc&chs=...
94,https://knowyourmeme.com/memes/diglett-undergr...,https://i.kym-cdn.com/photos/images/newsfeed/0...
...,...,...
12335,https://knowyourmeme.com/memes/needs-more-cowbell,https://i.kym-cdn.com/photos/images/newsfeed/0...
12417,https://knowyourmeme.com/memes/scary-prank-rea...,http://www.google.com/trends/viz?q=%22scary+ma...
12463,https://knowyourmeme.com/memes/dat-ass,http://i.imgur.com/aqmZf.jpg]
12561,https://knowyourmeme.com/memes/ate-my-balls,http://web.archive.org/web/19990222092144/http...


In [338]:
def explode_img_links(ImageColumn):
    Images = df[["URL",ImageColumn]]
    Images.columns=["URL","Images"]
    imglinks = Images.apply(lambda row: str(row["Images"]).replace("[","").replace("[","").split(","),axis=1).explode()
    imglinks = pd.DataFrame(imglinks,columns=["Link"])
    imglinks[imglinks["Link"]!='nan']
    imglinks["Link"] = imglinks.apply(lambda row: matched_string('http(...)',row["Link"]),axis=1)
    imglinks = imglinks[imglinks["Link"] != ""]
    Images = Images.join(imglinks)
    Images = Images.drop(labels=Images[Images["Link"].isna()].index)
    Images = Images[["URL","Link"]]
    return Images

In [343]:
ImageTables = {}
i=0
for col in ['AboutImages', 'OriginImages', 'SpreadImages', 'NotExamplesImages', 'SearchIntImages']:
    ImageTables[i] = explode_img_links(col)
    i += 1

In [344]:
ImageTables[0]

,URL,Link
11,https://knowyourmeme.com/memes/a-winner-is-you,https://i.kym-cdn.com/photos/images/newsfeed/0...
41,https://knowyourmeme.com/memes/do-it-faggot,https://i.kym-cdn.com/photos/images/newsfeed/0...
49,https://knowyourmeme.com/memes/cereal-guy,https://i.kym-cdn.com/photos/images/newsfeed/0...
56,https://knowyourmeme.com/memes/dr-hax,http://chart.apis.google.com/chart?cht=lc&chs=...
94,https://knowyourmeme.com/memes/diglett-undergr...,https://i.kym-cdn.com/photos/images/newsfeed/0...
...,...,...
12335,https://knowyourmeme.com/memes/needs-more-cowbell,https://i.kym-cdn.com/photos/images/newsfeed/0...
12417,https://knowyourmeme.com/memes/scary-prank-rea...,http://www.google.com/trends/viz?q=%22scary+ma...
12463,https://knowyourmeme.com/memes/dat-ass,http://i.imgur.com/aqmZf.jpg]
12561,https://knowyourmeme.com/memes/ate-my-balls,http://web.archive.org/web/19990222092144/http...


In [345]:
ImageTables[1]

,URL,Link
1,https://knowyourmeme.com/memes/roflcopter,https://i.kym-cdn.com/photos/images/newsfeed/0...
2,https://knowyourmeme.com/memes/bitches-dont-know,https://i.kym-cdn.com/photos/images/newsfeed/0...
4,https://knowyourmeme.com/memes/o-rly,https://i.kym-cdn.com/photos/images/newsfeed/0...
24,https://knowyourmeme.com/memes/rule-34,https://i.kym-cdn.com/photos/images/newsfeed/0...
29,https://knowyourmeme.com/memes/december-21st-2012,https://i.kym-cdn.com/photos/images/newsfeed/0...
...,...,...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,https://i.kym-cdn.com/photos/images/newsfeed/0...
12615,https://knowyourmeme.com/memes/vagina-bones,https://i.kym-cdn.com/photos/images/newsfeed/0...
12617,https://knowyourmeme.com/memes/liberal-douche-...,https://i.kym-cdn.com/photos/images/newsfeed/0...
12618,https://knowyourmeme.com/memes/polybius,https://i.kym-cdn.com/photos/images/newsfeed/0...


In [346]:
ImageTables[2]

,URL,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,https://i.kym-cdn.com/photos/images/newsfeed/0...
1,https://knowyourmeme.com/memes/roflcopter,https://i.kym-cdn.com/photos/images/newsfeed/0...
4,https://knowyourmeme.com/memes/o-rly,https://i.kym-cdn.com/photos/images/masonry/00...
6,https://knowyourmeme.com/memes/in-soviet-russia,https://i.kym-cdn.com/photos/images/newsfeed/0...
7,https://knowyourmeme.com/memes/domo,https://i.kym-cdn.com/photos/images/masonry/00...
...,...,...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,https://i.kym-cdn.com/photos/images/newsfeed/0...
12615,https://knowyourmeme.com/memes/vagina-bones,https://i.kym-cdn.com/photos/images/newsfeed/0...
12617,https://knowyourmeme.com/memes/liberal-douche-...,https://i.kym-cdn.com/photos/images/newsfeed/0...
12618,https://knowyourmeme.com/memes/polybius,https://i.kym-cdn.com/photos/images/masonry/00...


In [347]:
ImageTables[3]

,URL,Link
0,https://knowyourmeme.com/memes/this-is-relevan...,https://i.kym-cdn.com/photos/images/newsfeed/0...
2,https://knowyourmeme.com/memes/bitches-dont-know,https://i.kym-cdn.com/photos/images/newsfeed/0...
24,https://knowyourmeme.com/memes/rule-34,https://i.kym-cdn.com/photos/images/masonry/00...
27,https://knowyourmeme.com/memes/translator-san-...,https://i.kym-cdn.com/photos/images/newsfeed/0...
28,https://knowyourmeme.com/memes/e3-sony-2006-gi...,https://i.kym-cdn.com/photos/images/list/000/0...
...,...,...
12599,https://knowyourmeme.com/memes/one-million-fac...,https://i.kym-cdn.com/photos/images/masonry/00...
12600,https://knowyourmeme.com/memes/unicorning,https://i.kym-cdn.com/photos/images/masonry/00...
12603,https://knowyourmeme.com/memes/hamster-butts,https://i.kym-cdn.com/photos/images/masonry/00...
12614,https://knowyourmeme.com/memes/calvin-and-hobb...,https://i.kym-cdn.com/photos/images/newsfeed/0...


In [348]:
ImageTables[4]

,URL,Link
4803,https://knowyourmeme.com/memes/beyonce-always-...,https://i.kym-cdn.com/photos/images/newsfeed/0...
4900,https://knowyourmeme.com/memes/kylie-jenner-ch...,https://i.kym-cdn.com/photos/images/newsfeed/0...
5008,https://knowyourmeme.com/memes/villanova-cryin...,https://i.kym-cdn.com/photos/images/newsfeed/0...
5155,https://knowyourmeme.com/memes/selfie-stick,http://rp2.abs-cbnnews.com/sites/default/files...
5603,https://knowyourmeme.com/memes/alejandro-sabel...,https://i.kym-cdn.com/photos/images/newsfeed/0...
10090,https://knowyourmeme.com/memes/4chumblr,http://www.google.com/trends/viz?q=4chan
11290,https://knowyourmeme.com/memes/x-for-dummies,http://i.imgur.com/FFuP0.png]
12114,https://knowyourmeme.com/memes/crabcore,https://i.kym-cdn.com/photos/images/newsfeed/0...


In [352]:
AboutImg = ImageTables[0]
AboutImg["Type"] = "About"
OriginImg = ImageTables[1]
OriginImg["Type"] = "Origin"
SpreadImg = ImageTables[2]
SpreadImg["Type"] = "Spread"
NotExImg = ImageTables[3]
NotExImg["Type"] = "Notable Example"
SearchIntImg = ImageTables[4]
SearchIntImg["Type"] = "Search Interest"
ImageTable = AboutImg.append(OriginImg).append(SpreadImg).append(NotExImg).append(SearchIntImg)
ImageTable.sort_values(by=["URL","Link"])

,URL,Link,Type
6955,https://knowyourmeme.com/memes/%CD%A1-%CD%9C%C...,https://i.kym-cdn.com/photos/images/newsfeed/0...,Spread
6955,https://knowyourmeme.com/memes/%CD%A1-%CD%9C%C...,https://i.kym-cdn.com/photos/images/newsfeed/0...,Origin
725,https://knowyourmeme.com/memes/%D0%B1%D1%83%D0...,https://i.kym-cdn.com/photos/images/newsfeed/0...,Origin
4604,https://knowyourmeme.com/memes/%D0%BD%D0%B5%D1...,https://i.kym-cdn.com/photos/images/newsfeed/0...,Origin
4604,https://knowyourmeme.com/memes/%D0%BD%D0%B5%D1...,https://i.kym-cdn.com/photos/images/newsfeed/0...,Spread
...,...,...,...
3638,https://knowyourmeme.com/memes/zucked,https://i.kym-cdn.com/photos/images/newsfeed/0...,Spread
561,https://knowyourmeme.com/memes/zuckerberg-note...,https://i.kym-cdn.com/photos/images/small/000/...,Notable Example
1547,https://knowyourmeme.com/memes/zukos-honor,https://i.kym-cdn.com/photos/images/newsfeed/0...,Origin
1547,https://knowyourmeme.com/memes/zukos-honor,https://i.kym-cdn.com/photos/images/newsfeed/0...,Spread


### Type

In [368]:
Types = df[["URL","Type"]]
Types.columns = ["URL","Types"]
types = Types.apply(lambda row: str(row['Types']).replace("[","").replace("]","").split(","),axis=1).explode()
types = pd.DataFrame(types,columns=["Type"])
types["Type"] = types.apply(lambda row: str(row["Type"]).strip(),axis=1)
Types = Types.join(types)
Types = Types[Types["Type"]!='nan']
Types = Types[["URL","Type"]]
Types

,URL,Type
1,https://knowyourmeme.com/memes/roflcopter,https://knowyourmeme.com/types/slang
2,https://knowyourmeme.com/memes/bitches-dont-know,https://knowyourmeme.com/types/exploitable
6,https://knowyourmeme.com/memes/in-soviet-russia,https://knowyourmeme.com/types/snowclone
7,https://knowyourmeme.com/memes/domo,https://knowyourmeme.com/types/character
8,https://knowyourmeme.com/memes/i-like-turtles,https://knowyourmeme.com/types/viral-video
...,...,...
12609,https://knowyourmeme.com/memes/turnt,https://knowyourmeme.com/types/slang
12610,https://knowyourmeme.com/memes/roof-koreans,https://knowyourmeme.com/types/slang
12611,https://knowyourmeme.com/memes/knuckles--2,https://knowyourmeme.com/types/character
12615,https://knowyourmeme.com/memes/vagina-bones,https://knowyourmeme.com/types/slang


In [364]:
types.groupby(by="Type").count()

""
Type
https://knowyourmeme.com/types/advertisement
https://knowyourmeme.com/types/animal
https://knowyourmeme.com/types/axiom
https://knowyourmeme.com/types/catchphrase
https://knowyourmeme.com/types/character
...
https://knowyourmeme.com/types/sound-effect
https://knowyourmeme.com/types/viral-debate
https://knowyourmeme.com/types/viral-video


### Tags

In [429]:
Tags = df[['URL','Tags']]
tags = Tags.apply(lambda row: row['Tags'].replace("[","").replace("]","").split(","),axis=1).explode()
tags = pd.DataFrame(tags,columns=['Tag'])
tags["Tag"]=tags.apply(lambda row: row["Tag"].strip(),axis=1)
tags = tags[tags["Tag"]!=""]
Tags = Tags.join(tags)
Tags = Tags.drop(labels=Tags[Tags["Tag"].isna()].index)
Tags = Tags.drop_duplicates()
Tags = Tags[["URL","Tag"]]
Tags


,URL,Tag
0,https://knowyourmeme.com/memes/this-is-relevan...,image macros
0,https://knowyourmeme.com/memes/this-is-relevan...,comment
0,https://knowyourmeme.com/memes/this-is-relevan...,comments
0,https://knowyourmeme.com/memes/this-is-relevan...,approval
0,https://knowyourmeme.com/memes/this-is-relevan...,image macro
...,...,...
12620,https://knowyourmeme.com/memes/chocobo-racing-...,video game
12620,https://knowyourmeme.com/memes/chocobo-racing-...,youtube poop
12620,https://knowyourmeme.com/memes/chocobo-racing-...,ytp
12620,https://knowyourmeme.com/memes/chocobo-racing-...,square enix


In [425]:
toptags = pd.DataFrame(Tags.groupby(by="Tag").count().sort_values(by="URL",ascending=False)[:50],columns=["Tag"])
toptag = list(toptags["Tag"].index)

In [428]:
tops = {}
for i,top in enumerate(toptag):
    tops[i] = Tags.apply(lambda row: matched_pattern('(...)'+top+'(...)',str(row["Tag"])),axis=1)

### Enrichment idea: download more json files - broken??!!

In [1]:
import json
import numpy as np
import pandas as pd

# load the json file:
with open('kym_spotlight.json','r') as f:
    data = json.loads(f.read())

# import the json file to the pandas df
df2 = pd.json_normalize(data)

# shape of the df is enourmous
print(df2.shape)

# look the columnnames.. columns "content._" are noisy!
list(df2.columns)

(1, 53066)


['https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@text',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@confidence',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@support',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@types',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@sparql',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.@policy',
 'https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.Resources',
 'https://knowyourmeme.com/memes/roflcopter.@text',
 'https://knowyourmeme.com/memes/roflcopter.@confidence',
 'https://knowyourmeme.com/memes/roflcopter.@support',
 'https://knowyourmeme.com/memes/roflcopter.@types',
 'https://knowyourmeme.com/memes/roflcopter.@sparql',
 'https://knowyourmeme.com/memes/roflcopter.@policy',
 'https://knowyourmeme.com/memes/roflcopter.Resources',
 'https://knowyourmeme.com/memes/bitches-dont-know.@text',
 'https://knowyourmeme.

In [38]:
df2.head()

,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.labelAnnotations,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.safeSearchAnnotation.adult,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.safeSearchAnnotation.spoof,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.safeSearchAnnotation.medical,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.safeSearchAnnotation.violence,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.safeSearchAnnotation.racy,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.webDetection.webEntities,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.webDetection.fullMatchingImages,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.webDetection.partialMatchingImages,https://knowyourmeme.com/memes/this-is-relevant-to-my-interests.webDetection.pagesWithMatchingImages,...,https://knowyourmeme.com/memes/unicorning.safeSearchAnnotation.adult,https://knowyourmeme.com/memes/unicorning.safeSearchAnnotation.spoof,https://knowyourmeme.com/memes/unicorning.safeSearchAnnotation.medical,https://knowyourmeme.com/memes/unicorning.safeSearchAnnotation.violence,https://knowyourmeme.com/memes/unicorning.safeSearchAnnotation.racy,https://knowyourmeme.com/memes/unicorning.webDetection.webEntities,https://knowyourmeme.com/memes/unicorning.webDetection.fullMatchingImages,https://knowyourmeme.com/memes/unicorning.webDetection.partialMatchingImages,https://knowyourmeme.com/memes/unicorning.webDetection.pagesWithMatchingImages,https://knowyourmeme.com/memes/unicorning.webDetection.bestGuessLabels
0,"[{'mid': '/m/01yrx', 'description': 'Cat', 'sc...",VERY_UNLIKELY,VERY_LIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,"[{'entityId': '/m/04wpw', 'score': 0.7058, 'de...",[{'url': 'https://eigenblogger.files.wordpress...,[{'url': 'https://cdn2.psychologytoday.com/ass...,[{'url': 'https://knowyourmeme.com/memes/this-...,...,VERY_UNLIKELY,POSSIBLE,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,"[{'entityId': '/m/025rxrz', 'score': 0.7166, '...",[{'url': 'https://cache.cosmopolitan.fr/data/p...,[{'url': 'https://conteudo.imguol.com.br/c/not...,[{'url': 'https://knowyourmeme.com/memes/unico...,"[{'label': 'mascara de unicornio meme', 'langu..."
